In [1]:
from datetime import date, timedelta
today=date.today()
days_before=(date.today()-timedelta(days=30))
d=today.strftime("%m-%d-%y")
d

'04-12-20'

In [2]:
cnn_url='https://edition.cnn.com/world/live-news/coronavirus-outbreak-{}-intl-hnk/index.html'.format(d)
from bs4 import BeautifulSoup
import requests
html=requests.get(cnn_url).text
cnn_url

'https://edition.cnn.com/world/live-news/coronavirus-outbreak-04-12-20-intl-hnk/index.html'

In [3]:
soup=BeautifulSoup(html)
soup.find('link')['href']

'/cnn-live-story/static/favicon.ico'

In [4]:
import spacy
from spacy import load
nlp=spacy.load('en_core_web_sm')

In [5]:
for link in soup.find_all("h2"):
    head_line=("Headline : {}".format(link.text))
    for ent in nlp(link.text).ents:
        text_entity="\tText: {},Entity : {}".format(ent.text,ent.label_)

        
    

In [6]:
nbc_url ='https://www.nbcnews.com/health/coronavirus'
cnbc_rss_url ='https://www.cnbc.com/id/10000108/device/rss/rss.html'

# collecting the data from 3 websites

In [7]:

urls=[cnn_url, nbc_url, cnbc_rss_url]
formats=['html.parser','html.parser','xml']
tags = ['h2','h2','description']
website = ['CNN','NBC','CNBC']


In [8]:
#crawling all the pages
crawl_len =0
news_dict=[]
for url in urls:
    print("Crawling web page..........{}".format(url))
    response =requests.get(url)
    soup = BeautifulSoup(response.content, formats[crawl_len])
    for link in soup.find_all(tags[crawl_len]):
        if len(link.text.split(" "))>4:
            print("Headline : {}".format(link.text))
            #entities=[]
            entities=[(ent.text,ent.label_) for ent in nlp(link.text).ents]
                #print("\tText: {},Entity : {}".format(ent.text,ent.label_))
            news_dict.append({"Website":website[crawl_len],'URL':url,'Headline':link.text,'entities':entities})
    crawl_len+=1
        
        

Crawling web page..........https://edition.cnn.com/world/live-news/coronavirus-outbreak-04-12-20-intl-hnk/index.html
Crawling web page..........https://www.nbcnews.com/health/coronavirus
Headline : U.S. now leads the world in coronavirus deaths, surpassing Italy
Headline : Live Blog / Latest news on the global pandemic
Headline : Doctor reunites with wife and new baby after work with coronavirus 'dirty team'
Headline : The woman leading COVID-19 vaccine trials is 'not your average pocket-protector scientist'
Headline : U.K.'s Boris Johnson leaves hospital after personal COVID-19 battle
Headline : One year waiting: Transgender man who wants to enlist cites coronavirus urgency
Headline : Coronavirus could 'decimate' Latino wealth, hammered by the Great Recession
Headline : Vietnamese-owned nail salons donate thousands of masks, gloves, more to hospitals
Headline : 'Saturday Night Live' practices social distancing
Headline : 'Cannonball' coast-to-coast drive record set amid virus shutdown

Headline : A New Jersey company allegedly tried to sell $45 million of respirator masks at inflated prices.
Headline : The U.K. government has urged everyone to stay at home over the Easter weekend.
Headline : UnitedHealth Group was hired to help process and expedite the initial $30 billion tranche of payments.
Headline : New York saw more than 700 deaths from Covid-19 on Thursday, Gov. Andrew Cuomo said.
Headline : Lifting stay-at-home orders too soon could lead to a "deadly resurgence" and there's been an "alarming acceleration" of cases in other countries, WHO officials said.
Headline : Google says it has seen a dramatic rise in telemedicine in the midst of the COVID-19 pandemic, because people would prefer to see a doctor at home.
Headline : With strict stay-at-home orders, social distancing measures and countrywide lockdowns, the faithful and religious leaders have had to find creative ways to bring their services and communities together in these trying times.
Headline : Black pe

In [55]:
news_dict

SyntaxError: invalid syntax (<ipython-input-55-ec70a0a9b22f>, line 1)

# Converting to dataframe

In [56]:
import pandas as pd
df=pd.DataFrame(news_dict)


#In the last column(feautre) we got a list, splitting this list and concating with dataframe

In [57]:

df=pd.concat([df[['Website','URL','Headline']],df['entities'].apply(pd.Series)],axis=1)

In [58]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
#importing the vader sentiment analyzer
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\New\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [59]:
sid = SentimentIntensityAnalyzer()

In [69]:
df['sentiment'] =df['Headline'].apply(lambda x:sid.polarity_scores(x))
df.to_csv('sentiment_analysis.csv')
df=pd.concat([df[['Website','URL','Headline']],df['sentiment'].apply(pd.Series)],axis=1)
df

,Website,URL,Headline,neg,neu,pos,compound
0,NBC,https://www.nbcnews.com/health/coronavirus,U.S. now leads the world in coronavirus deaths...,0.000,1.000,0.000,0.0000
1,NBC,https://www.nbcnews.com/health/coronavirus,Live Blog / Latest news on the global pandemic,0.000,1.000,0.000,0.0000
2,NBC,https://www.nbcnews.com/health/coronavirus,Doctor reunites with wife and new baby after w...,0.195,0.805,0.000,-0.4404
3,NBC,https://www.nbcnews.com/health/coronavirus,The woman leading COVID-19 vaccine trials is '...,0.000,1.000,0.000,0.0000
4,NBC,https://www.nbcnews.com/health/coronavirus,U.K.'s Boris Johnson leaves hospital after per...,0.245,0.755,0.000,-0.3818
...,...,...,...,...,...,...,...
96,CNBC,https://www.cnbc.com/id/10000108/device/rss/rs...,"With strict stay-at-home orders, social distan...",0.000,0.838,0.162,0.7003
97,CNBC,https://www.cnbc.com/id/10000108/device/rss/rs...,Black people make up about 14% of Michigan's p...,0.000,1.000,0.000,0.0000
98,CNBC,https://www.cnbc.com/id/10000108/device/rss/rs...,"COVID-19 has infected more than 1,650,200 peop...",0.322,0.678,0.000,-0.8225
99,CNBC,https://www.cnbc.com/id/10000108/device/rss/rs...,U.K. officials are considering an exit strateg...,0.000,0.927,0.073,0.2263


In [63]:
df.to_csv('final_data.csv')

In [24]:
import googlemaps
